In [ ]:
import cv2
import pickle
import cvzone
import numpy as np

# Video feed
cap = cv2.VideoCapture('video.mp4')

with open('CarParkPos', 'rb') as f:
    posList = pickle.load(f)

width, height = 470, 180  # Same width and height

# Set consistent frame size
frame_width = 720
frame_height = 1080

def checkParkingSpace(imgPro, img):
    spaceCounter = 0

    for pos in posList:
        x, y = pos

        # Ensure the coordinates remain within the frame bounds
        x_end = min(x + width, frame_width)
        y_end = min(y + height, frame_height)

        imgCrop = imgPro[y:y_end, x:x_end]
        count = cv2.countNonZero(imgCrop)

        if count < 1200:
            color = (0, 255, 0)
            thickness = 5
            spaceCounter += 1
        else:
            color = (0, 0, 255)
            thickness = 2

        cv2.rectangle(img, (x, y), (x_end, y_end), color, thickness)
        cvzone.putTextRect(img, str(count), (x, y + height - 3), scale=1,
                           thickness=2, offset=0, colorR=color)

    cvzone.putTextRect(img, f'Free: {spaceCounter}/{len(posList)}', (50, 50), scale=2,
                       thickness=4, offset=10, colorR=(0, 200, 0))

cv2.namedWindow("Image", cv2.WINDOW_NORMAL)  # Allow resizing
cv2.resizeWindow("Image", frame_width, frame_height)  # Set fixed size

while True:
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
    success, img = cap.read()
    
    if not success:
        break

    # Resize the video frame to the consistent size
    img = cv2.resize(img, (frame_width, frame_height))

    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 1)
    imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY_INV, 25, 16)
    imgMedian = cv2.medianBlur(imgThreshold, 5)
    kernel = np.ones((3, 3), np.uint8)
    imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)

    checkParkingSpace(imgDilate, img)

    cv2.imshow("Image", img)
    
    key = cv2.waitKey(10) & 0xFF  # Read key input
    if key == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import pickle
import cvzone
import numpy as np
import serial
import time

# Initialize Serial Communication with Arduino (Update COM Port)
arduino = serial.Serial('COM9', 9600, timeout=1)
time.sleep(2)  # Wait for connection

# Use phone's camera as live feed (Replace with actual phone IP Webcam URL)
url = "http://192.168.1.10:8080/video"
cap = cv2.VideoCapture(url)

with open('CarParkPos', 'rb') as f:
    posList = pickle.load(f)

width, height = 280, 140  # Parking slot dimensions
frame_width, frame_height = 720, 1080  # Video frame size

def send_to_arduino(slots):
    message = f"{slots}\n"  # Ensure newline for proper reading
    arduino.write(message.encode())

def checkParkingSpace(imgPro, img):
    spaceCounter = 0

    for pos in posList:
        x, y = pos
        x_end = min(x + width, frame_width)
        y_end = min(y + height, frame_height)

        imgCrop = imgPro[y:y_end, x:x_end]
        count = cv2.countNonZero(imgCrop)

        if count < 1000:
            color = (0, 255, 0)
            thickness = 5
            spaceCounter += 1
        else:
            color = (0, 0, 255)
            thickness = 2

        cv2.rectangle(img, (x, y), (x_end, y_end), color, thickness)
        cvzone.putTextRect(img, str(count), (x, y + height - 3), scale=1,
                           thickness=2, offset=0, colorR=color)

    cvzone.putTextRect(img, f'Free: {spaceCounter}/{len(posList)}', (50, 50), scale=2,
                       thickness=4, offset=10, colorR=(0, 200, 0))
    
    # Send available slots count to Arduino
    send_to_arduino(spaceCounter)

cv2.namedWindow("Image", cv2.WINDOW_NORMAL)  # Allow resizing
cv2.resizeWindow("Image", frame_width, frame_height)  # Set fixed size

while True:
    success, img = cap.read()
    
    if not success:
        print("Failed to grab frame")
        break

    img = cv2.resize(img, (frame_width, frame_height))
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 1)
    imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY_INV, 25, 16)
    imgMedian = cv2.medianBlur(imgThreshold, 5)
    kernel = np.ones((3, 3), np.uint8)
    imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)

    checkParkingSpace(imgDilate, img)
    cv2.imshow("Image", img)
    
    key = cv2.waitKey(1) & 0xFF  # Faster frame processing
    if key == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()
